In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31751
2,Huelva,Confirmados PDIA 14 días,1508
3,Huelva,Tasa PDIA 14 días,"293,8597345908763"
4,Huelva,Confirmados PDIA 7 días,756
5,Huelva,Total Confirmados,31931
6,Huelva,Curados,29228
7,Huelva,Fallecidos,383


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31751.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8491.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 488 personas en los últimos 7 días 

Un positivo PCR cada 248 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31751.0,756.0,1508.0,513170.0,147.319602,293.859735,272.0
Huelva-Costa,18269.0,410.0,817.0,289548.0,141.600011,282.163924,154.0
Huelva (capital),8491.0,295.0,581.0,143837.0,205.093265,403.929448,110.0
Condado-Campiña,10322.0,255.0,546.0,156231.0,163.219848,349.482497,95.0
Sierra de Huelva-Andévalo Central,2811.0,88.0,140.0,67391.0,130.581235,207.742874,23.0
Bollullos Par del Condado,992.0,21.0,47.0,14387.0,145.965107,326.683812,16.0
San Juan del Puerto,520.0,25.0,43.0,9411.0,265.646584,456.912124,16.0
Aljaraque,1120.0,43.0,80.0,21474.0,200.242153,372.543541,13.0
Punta Umbría,825.0,21.0,43.0,15355.0,136.763269,280.039075,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Niebla,239.0,8.0,28.0,4116.0,194.363460,680.272109,3.0
Villarrasa,177.0,10.0,14.0,2211.0,452.284034,633.197648,5.0
Valverde del Camino,597.0,45.0,68.0,12750.0,352.941176,533.333333,11.0
Palos de la Frontera,679.0,37.0,60.0,11742.0,315.108159,510.986203,13.0
Almonte,1842.0,49.0,112.0,24507.0,199.942873,457.012282,6.0
San Juan del Puerto,520.0,25.0,43.0,9411.0,265.646584,456.912124,16.0
Palma del Condado (La),1160.0,18.0,45.0,10801.0,166.651236,416.628090,8.0
Huelva (capital),8491.0,295.0,581.0,143837.0,205.093265,403.929448,110.0
Moguer,1304.0,34.0,86.0,21867.0,155.485435,393.286688,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),146.0,1.0,8.0,2327.0,42.973786,343.790288,0.0,0.125000
Beas,211.0,1.0,6.0,4341.0,23.036167,138.217001,0.0,0.166667
Lucena del Puerto,254.0,2.0,9.0,3261.0,61.330880,275.988960,0.0,0.222222
Lepe,2134.0,7.0,26.0,27880.0,25.107604,93.256815,6.0,0.269231
Isla Cristina,2150.0,6.0,21.0,21393.0,28.046557,98.162950,0.0,0.285714
Niebla,239.0,8.0,28.0,4116.0,194.363460,680.272109,3.0,0.285714
Calañas,191.0,2.0,6.0,2768.0,72.254335,216.763006,0.0,0.333333
Ayamonte,1320.0,4.0,11.0,21104.0,18.953753,52.122820,3.0,0.363636
Gibraleón,632.0,10.0,26.0,12737.0,78.511423,204.129701,0.0,0.384615
